In [51]:
import numpy as np
import pandas as pd

In [52]:
hydrate=pd.read_csv('s2-hydrate.csv',index_col=0)
cages=pd.read_csv('s2-cages.csv')
thf=pd.read_csv('thf.csv')

Refactor the hydrate file into an xyz format

In [53]:
hydrate_ox=pd.DataFrame()
hydrate_ox[['X','Y','Z']]=hydrate[['OX','OY','OZ']]
hydrate_ox.insert(0,'ELEMENT','O')
hydrate_hy=pd.DataFrame()
hydrate_hy[['X','Y','Z']]=hydrate[['H1X','H1Y','H1Z']]
tmp=pd.DataFrame()
tmp[['X','Y','Z']]=hydrate[['H2X','H2Y','H2Z']]
hydrate_hy=pd.concat([hydrate_hy,tmp])
hydrate_hy.insert(0,'ELEMENT','H')
hydrate_total = pd.concat([hydrate_ox[['ELEMENT','X','Y','Z']],hydrate_hy[['ELEMENT','X','Y','Z']]]).reset_index().drop(columns='index')

In [54]:
hydrate_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ELEMENT  408 non-null    object 
 1   X        408 non-null    float64
 2   Y        408 non-null    float64
 3   Z        408 non-null    float64
dtypes: float64(3), object(1)
memory usage: 12.9+ KB


The following allows to visualize the hydrate in 3D (not as fancy as VMD though)

In [55]:
%matplotlib qt5

In [56]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=hydrate_ox['X'].values,ys=hydrate_ox['Y'].values,zs=hydrate_ox['Z'].values,c='red')
ax.scatter(xs=hydrate_hy['X'].values,ys=hydrate_hy['Y'].values,zs=hydrate_hy['Z'].values,c='white')

In [57]:
thf_center = thf[['X', 'Y', 'Z']].mean()
thf_center

X    17.677534
Y   -11.329676
Z     0.000000
dtype: float64

In [58]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=thf['X'].values,ys=thf['Y'].values,zs=thf['Z'].values,c='red')
ax.scatter(xs=thf_center['X'],ys=thf_center['Y'],zs=thf_center['Z'],c='blue')
plt.show()

In [59]:
thf_translated = thf.copy()
large_cages = cages[cages['TYPE'].str.contains('Large')]
large_cages[['X','Y','Z']].iloc[0]

X    8.651
Y    8.655
Z    8.654
Name: 16, dtype: float64

In [60]:
dxyz = thf_center - large_cages[['X','Y','Z']].iloc[0]
dxyz

X     9.026534
Y   -19.984676
Z    -8.654000
dtype: float64

In [61]:
thf_translated = thf_translated[['X','Y','Z']] - dxyz
thf_translated

,X,Y,Z
0,7.848466,9.359676,8.654
1,7.848466,7.859676,8.654
2,9.275066,7.396176,8.654
3,10.156766,8.609676,8.654
4,9.275066,9.823176,8.654
5,6.982826,9.988616,8.654
6,8.015866,10.416496,8.654
7,6.982826,7.230736,8.654
8,8.015866,6.802856,8.654
9,9.605716,6.378546,8.654


In [62]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=hydrate_ox['X'].values,ys=hydrate_ox['Y'].values,zs=hydrate_ox['Z'].values,c='red')
ax.scatter(xs=hydrate_hy['X'].values,ys=hydrate_hy['Y'].values,zs=hydrate_hy['Z'].values,c='white')
ax.scatter(xs=thf_translated['X'].values,ys=thf_translated['Y'].values,zs=thf_translated['Z'].values,c='blue')
plt.show()

Cool, so now THF is centered within the large cell of the hydrate.

In [81]:
# This loop recenters THF and places it in all 8 large cages of the sII hydrate
thf_hydrate = hydrate_total.copy()
for i in range(len(large_cages.index)):
    dxyz = thf_center - large_cages[['X','Y','Z']].iloc[i]
    thf_translated = thf.copy()
    thf_translated[['X','Y','Z']] = thf_translated[['X','Y','Z']] - dxyz
    thf_hydrate = pd.concat([thf_hydrate, thf_translated])

thf_hydrate.info()

    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512 entries, 0 to 12
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ELEMENT  512 non-null    object 
 1   X        512 non-null    float64
 2   Y        512 non-null    float64
 3   Z        512 non-null    float64
dtypes: float64(3), object(1)
memory usage: 20.0+ KB


In [83]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=thf_hydrate['X'].loc[thf_hydrate['ELEMENT'].str.contains('O')].values,ys=thf_hydrate['Y'].loc[thf_hydrate['ELEMENT'].str.contains('O')].values,zs=thf_hydrate['Z'].loc[thf_hydrate['ELEMENT'].str.contains('O')].values,c='red')
ax.scatter(xs=thf_hydrate['X'].loc[thf_hydrate['ELEMENT'].str.contains('H')].values,ys=thf_hydrate['Y'].loc[thf_hydrate['ELEMENT'].str.contains('H')].values,zs=thf_hydrate['Z'].loc[thf_hydrate['ELEMENT'].str.contains('H')].values,c='white')
ax.scatter(xs=thf_hydrate['X'].loc[thf_hydrate['ELEMENT'].str.contains('C')].values,ys=thf_hydrate['Y'].loc[thf_hydrate['ELEMENT'].str.contains('C')].values,zs=thf_hydrate['Z'].loc[thf_hydrate['ELEMENT'].str.contains('C')].values,c='blue')
plt.show()

In [124]:
thf_hydrate.to_csv('tmp.xyz', sep='\t', index=False)